<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/CLASSIFICATION2_roberta_LoRA_stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate
!pip install torch tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# 멀티 레이블 데이터셋 로드
sof_dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/MultiLabel/rev_tag_training_samples.csv',
    split='train'
)

# Title과 Body를 결합하여 Body 업데이트
def update_body(example):
    updated_body = example['Title'] + ' ' + example['Body']
    example['Body'] = updated_body
    return example

# map 함수를 사용하여 모든 데이터에 update_body 함수 적용
sof_dataset_train = sof_dataset_train.map(update_body)

# # 변경된 데이터셋 출력
# print(sof_dataset_train['Title'][0])
# print(sof_dataset_train['Body'][0])
print(sof_dataset_train)

# 레이블 이름으로 구성된 리스트
labels = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

# 레이블 이름을 키로 하고 카운터를 값으로 하는 딕셔너리 생성
label_counter = {label: 0 for label in labels}

for example in sof_dataset_train:
    # 각 샘플에서 0과 1로 표현된 레이블 상태를 이용해 카운트를 업데이트합니다.
    for label in labels:
        if example[label] == 1:
            label_counter[label] += 1

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset({
    features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 20000
})
{'Algorithms': 4225, 'Backend': 3723, 'Data Science': 3487, 'Databases': 3072, 'Dev Tools': 3655, 'Frontend': 3966, 'Mobile': 2683, 'Systems': 3996, 'iOS/macOS': 2587}


In [3]:
from datasets import load_dataset

# 멀티 레이블 데이터셋 로드
sof_dataset_val = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/MultiLabel/rev_tag_validation_samples.csv',
    split='train'
)

# Title과 Body를 결합하여 Body 업데이트
def update_body(example):
    updated_body = example['Title'] + ' ' + example['Body']
    example['Body'] = updated_body
    return example

# map 함수를 사용하여 모든 데이터에 update_body 함수 적용
sof_dataset_val = sof_dataset_val.map(update_body)

# print(sof_dataset_val['Title'][0])
# print(sof_dataset_val['Body'][0])
print(sof_dataset_val)

# 레이블 이름으로 구성된 리스트
labels = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

# 레이블 이름을 키로 하고 카운터를 값으로 하는 딕셔너리 생성
label_counter = {label: 0 for label in labels}

for example in sof_dataset_val:
    # 각 샘플에서 0과 1로 표현된 레이블 상태를 이용해 카운트를 업데이트합니다.
    for label in labels:
        if example[label] == 1:
            label_counter[label] += 1

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

Dataset({
    features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 6500
})
{'Algorithms': 1573, 'Backend': 722, 'Data Science': 1519, 'Databases': 230, 'Dev Tools': 656, 'Frontend': 1821, 'Mobile': 167, 'Systems': 1178, 'iOS/macOS': 28}


# Tokenizing

In [4]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict

sof_dataset= DatasetDict({
    "train": sof_dataset_train,
    "test": sof_dataset_val
})

# DatasetDict 출력
print(sof_dataset)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

# 토크나이즈 함수 정의
def tokenize_function(examples):
    return tokenizer(examples["Body"], padding="max_length", truncation=True, max_length=128)

# 데이터셋 토크나이즈
tokenized_datasets = sof_dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)

def transform_labels(examples):
    labels = []
    for i in range(len(examples['Algorithms'])):  # 예제의 개수 만큼 반복
        label = [examples[c][i] for c in ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']]
        labels.append(list(map(float, label)))  # 레이블을 float 타입으로 변환
    return {"labels": labels}

tokenized_datasets = tokenized_datasets.map(transform_labels, batched=True)
print(tokenized_datasets)


DatasetDict({
    train: Dataset({
        features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 6500
    })
})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS', 'input_ids', 'attention_mask'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS', 'input_ids', 'attention_mask'],
        num_rows: 6500
    })
})


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6500
    })
})


# Model / LoRA config

In [5]:
from peft import LoraConfig, TaskType
from peft import get_peft_model
from transformers import AutoModelForSequenceClassification

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=8,lora_alpha=16, lora_dropout=0.1
)

# model = RobertaForSequenceClassification.from_pretrained(
#     'FacebookAI/roberta-large',
#     num_labels=5
# )

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/roberta-large",
    num_labels=9,  # 레이블의 개수 설정
    problem_type="multi_label_classification"
)

model = get_peft_model(model, lora_config)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
import torch
import evaluate

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # 이미 NumPy 배열인지 확인하고, 아니면 PyTorch 텐서로 변환
    if isinstance(logits, np.ndarray):
        logits = torch.from_numpy(logits)
    if isinstance(labels, np.ndarray):
        labels = torch.from_numpy(labels)

    # PyTorch 텐서에서 시그모이드 함수 적용 및 이진화
    predictions = torch.sigmoid(logits)
    predictions = (predictions > 0.5).int()

    # PyTorch 텐서를 CPU로 이동시키고 NumPy 배열로 변환
    predictions = predictions.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()

    # 평가 메트릭 계산
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, predictions, average='micro')

    # 평탄화된 정확도 계산
    flat_predictions = predictions.flatten()
    flat_labels = labels.flatten()
    flat_accuracy = accuracy_score(flat_labels, flat_predictions)

    # 평가 메트릭 결과를 딕셔너리 형태로 반환
    return {
        'flat_accuracy': flat_accuracy,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1_score
    }


# Fine-tuning

In [7]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(output_dir="/content/drive/MyDrive/LoRA/CLASSIFICATION2_roberta_LoRA_stackoverflow",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=10)


# 트레이너 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Flat Accuracy,Accuracy,Precision,Recall,F1
1,0.223800,0.166888,0.940171,0.587692,0.801786,0.739422,0.769342
2,0.196200,0.151462,0.945128,0.622308,0.795111,0.799341,0.797220
3,0.185000,0.140778,0.947590,0.633692,0.807751,0.802635,0.805185
4,0.178300,0.140111,0.947709,0.637692,0.800348,0.816063,0.808129
5,0.172300,0.135817,0.949402,0.645538,0.812358,0.812769,0.812563
6,0.163000,0.136178,0.949487,0.644923,0.803714,0.827844,0.815601
7,0.161800,0.136984,0.949504,0.648923,0.805240,0.825437,0.815213
8,0.152500,0.132925,0.950615,0.654615,0.809907,0.828477,0.819087
9,0.150400,0.135176,0.949812,0.647077,0.800412,0.836711,0.818159
10,0.147000,0.134569,0.950103,0.650923,0.803391,0.834431,0.818617


TrainOutput(global_step=25000, training_loss=0.17867727996826172, metrics={'train_runtime': 5821.6594, 'train_samples_per_second': 34.354, 'train_steps_per_second': 4.294, 'total_flos': 4.68811017216e+16, 'train_loss': 0.17867727996826172, 'epoch': 10.0})

In [9]:
# 트레이닝이 완료된 후 모델 저장
trainer.save_model("/content/drive/MyDrive/LoRA/CLASSIFICATION2_roberta_LoRA_stackoverflow")

# Inference

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 모델과 토크나이저 로드
model_path = "/content/drive/MyDrive/LoRA/CLASSIFICATION2_roberta_LoRA_stackoverflow"
model = AutoModelForSequenceClassification.from_pretrained(model_path,  num_labels=9 )
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

def predict_multilabel(text, model, tokenizer):
    # 텍스트 토크나이징
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # 모델 예측
    with torch.no_grad():
        logits = model(**inputs).logits

    # 시그모이드 함수 적용 및 이진화
    predictions = torch.sigmoid(logits)
    predictions = (predictions > 0.5).int()

    # NumPy 배열로 변환
    predictions = predictions.detach().cpu().numpy()

    return predictions

# 추론하고자 하는 샘플 텍스트
sample_text = "working of compareTo() method of Comparable interface            I have one Employee class and the requirement is to sort the objects using comparable interface. The output with this code is :The difference of this id and other id is..** 6  other id**1The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**1The difference of this id and other id is..** 11  other id**3The difference of this id and other id is..** 11  other id**6"

# 추론 실행
predictions = predict_multilabel(sample_text, model, tokenizer)

# 예측 결과 출력
labels = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']
predicted_labels = [label for prediction, label in zip(predictions[0], labels) if prediction == 1]
print(f"Predicted labels: {predicted_labels}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted labels: ['Algorithms']
